In [1]:
import pymongo
 
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [2]:
myclient.list_database_names()

['admin', 'config', 'local', 'tweet', 'tweet_project']

In [3]:
mydb = myclient["tweet_project"] # create a database test under the client

In [4]:
mydb.list_collection_names()

['origin_tweet', 'retweet_tweet', 'reply_tweet', 'quote_tweet']

In [5]:
mycol = mydb["origin_tweet"] 

In [6]:
# myquery = { "reply_list" : { "$exists": "false"}}
# mydoc = mycol.find(myquery).limit(3).sort('timestamp_ms', -1)

In [7]:
# for x in mydoc:
#     print(x)

In [8]:
col_reply = mydb["reply_tweet"]
col_quoted = mydb["quote_tweet"]
col_retweet = mydb["retweet_tweet"]

In [10]:
for x in col_reply.find().limit(1):
    print(x)

{'_id': ObjectId('64425b022b4f18030a636eb2'), 'tweet_id': '1254022773598572544', 'text': '@VinceMcMahon @TripleH We hereby honor to announce that we have developed a brand new treatment method using UV rays / Laser Ray that might be used effectively for preventing all mankind from infectious virus.\n\n#covid #covid19 #Covid_19 #corona #vascular #endovascular https://t.co/YkVlYtJiGL', 'hashtags': [{'text': 'covid', 'indices': [211, 217]}, {'text': 'covid19', 'indices': [218, 226]}, {'text': 'Covid_19', 'indices': [227, 236]}, {'text': 'corona', 'indices': [237, 244]}, {'text': 'vascular', 'indices': [245, 254]}, {'text': 'endovascular', 'indices': [255, 268]}], 'userid': '1091660129894838272', 'username': 'Milli teknoloji', 'user_screen_name': 'milliteknoloj', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'timestamp_ms': '1587817302499', 'reply_to_tweeet_id': '1253768179517657088', 'reply_to_user_id': '1222639789', 'reply_to_user_name': 'VinceMcMahon'}


In [ ]:
def complete_tweetid(self,):
    main_query = { "tweet_id" : {"$eq" : None }}
    main_query['tweet_id']['$eq'] = reply_id
    main_doc = list(col_reply.find(myquery_reply))
    
    wait_list = [] + main_doc[0]["reply_list"]
    content = [] + main_doc
    
    while len(wait_list)!=0:
        reply_id = wait_list[0]
        wait_list.pop(0)
        
        reply_query = { "tweet_id" : {"$eq" : None }}
        reply_query['tweet_id']['$eq'] = reply_id
        
        doc = list(col_reply.find(reply_query))
        wait_list += doc[0]["reply_list"]
        content += doc
    
    return content

In [37]:
# %%timeit pass #-n 4 -r 2    


rec = []
def keywordSear(searchString, criteria, sortMethod = "timestamp_ms", numofEntries = 1):
    if criteria in ["keyword", "hashtags"]:
        if criteria == "keyword":
            myquery = {"$text": {"$search": None}}
            myquery['$text']['$search'] = searchString
            #We can also use favorite_count for sortMethod
            mydoc = mycol.find(myquery).sort(sortMethod, -1).limit(numofEntries)
        if criteria == "hashtags":
            myquery = {"hashtags.text": {"$eq": None}}
            myquery['hashtags.text']['$eq'] = searchString
            mydoc = mycol.find(myquery).sort(sortMethod, -1).limit(numofEntries)
        #     elif criteria in ["user"]

    m = list(mydoc)
    for each_tweet in m:
        print("\n Orginal tweet is: \n", each_tweet)
        try:
            print("Replies to the above tweet are as follows")
            for each_reply in range(min(len(each_tweet["reply_list"]), 3)):
                x = recursive_reply(each_tweet["reply_list"][each_reply])
                print(x)
        except:
            print("Key missing for replies")

        try:
            print("Quoted tweets to the above tweet are as follows")
            for each_quote in range(min(len(each_tweet["quoted_list"]), 3)):
                x = quote_tweet(each_tweet["quoted_list"][each_quote])
                print(x)
        except:
            print("Key missing for quoted")

        try:
            print("Retweeted tweets to the above tweet are as follows")
            for each_retweet in range(min(len(each_tweet["retweeted_list"]), 3)):
                x = retweet_tweet(each_tweet["retweeted_list"][each_retweet])
                print(x)
        except:
            print("Key missing for retweets")
            
    if len(m) < 10:
        keywordSear_reply(searchString, numofEntries = 10-len(m) )
        
def recursive_reply(key):
    rec.append(reply_tweet(key))
    myquery_rec_reply = {"$tweet_id": {"$eq": None}}
    myquery_rec_reply['tweet_id']['$eq'] = key
    mydoc_rec_reply = list(col_reply.find(myquery_rec_reply))
    print(mydoc_rec_reply["reply_list"], len(mydoc_rec_reply["reply_list"]), "sammmm")
    for each_reply in range(min(len(mydoc_rec_reply["reply_list"]), 3)):
                x = recursive_reply(each_tweet["reply_list"][each_reply])
                print(x)
    for i in list(mydoc_rec_reply[0]["reply_list"]):
        reply_tweet(i)

            
def reply_tweet(reply_id):
    myquery_reply = { "tweet_id" : {"$eq" : None }}
    myquery_reply['tweet_id']['$eq'] = reply_id
    mydoc_reply = col_reply.find(myquery_reply)
    sam = list(mydoc_reply)[0]
    print("sam for rep", sam)
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]

def keywordSear_reply(searchStr, numofEntries):
    criteria = "keyword"
    sortMethod = "timestamp_ms"
    if criteria == "keyword":
        myquery = {"$text": {"$search": None}}
        myquery['$text']['$search'] = searchStr
            #We can also use favorite_count for sortMethod
        mydoc = col_reply.find(myquery).sort(sortMethod, -1).limit(numofEntries)
    z = list(mydoc)
    for each_tweet in z:
        print("\n Search in reply yields: \n", each_tweet)

        
def quote_tweet(quote_id):
    myquery_quote = { "tweet_id" : {"$eq" : None }}
    myquery_quote['tweet_id']['$eq'] = quote_id
    mydoc_quote = col_quoted.find(myquery_quote)
    sam = list(mydoc_quote)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]

def retweet_tweet(retweet_id):
    myquery_retweet = { "tweet_id" : {"$eq" : None }}
    myquery_retweet['tweet_id']['$eq'] = retweet_id
    mydoc_retweet = col_retweet.find(myquery_retweet)
    sam = list(mydoc_retweet)[0]
    return [sam["tweet_id"], sam["text"], sam["username"], sam["favorite_count"]]



mydoc = keywordSear("covid", "keyword")
# mydoc = keywordSear("covid", "hashtags")
# ["tweet_id"], 
#             each_tweet["text"], each_tweet["hashtags"], each_tweet["username"],each_tweet["quote_count"], 
#             each_tweet["reply_count"], each_tweet["retweet_count"], each_tweet["favorite_count"]




 Orginal tweet is: 
 {'_id': ObjectId('64425e032b4f18030a63f168'), 'tweet_id': '1254009056517332998', 'text': 'And where is the evidence that Covid 19 is easily spread outdoors?  https://t.co/YPcJXU1uqw', 'hashtags': [], 'userid': '112047805', 'username': 'Brit Hume', 'user_screen_name': 'brithume', 'quote_count': 132, 'reply_count': 477, 'retweet_count': 2226, 'favorite_count': 7300, 'timestamp_ms': '1587826112422', 'quoted_list': ['1254059423842144256', '1254058980319494144', '1254058766317817856', '1254058479649730561', '1254058016652947457', '1254057991181082624', '1254057779892858881', '1254057313150947331', '1254055939063402504', '1254055652927909890', '1254055025472450562', '1254054396519948290', '1254054294363484161', '1254054152541478913', '1254053659186323456', '1254053591947632642', '1254053492597063682', '1254053492274212865', '1254053286149328896', '1254052839061704711', '1254052837283291137', '1254052532764045312', '1254052526552481792', '1254051937118535680', '125405149

In [101]:
# myquery = { "favorite_count" : { "$ne": "0"}}
# mydoc = col_reply.find(myquery).limit(5)

In [102]:
# for x in mydoc:
#     print(x)

{'_id': ObjectId('64425b022b4f18030a636eb2'), 'tweet_id': '1254022773598572544', 'text': '@VinceMcMahon @TripleH We hereby honor to announce that we have developed a brand new treatment method using UV rays / Laser Ray that might be used effectively for preventing all mankind from infectious virus.\n\n#covid #covid19 #Covid_19 #corona #vascular #endovascular https://t.co/YkVlYtJiGL', 'hashtags': [{'text': 'covid', 'indices': [211, 217]}, {'text': 'covid19', 'indices': [218, 226]}, {'text': 'Covid_19', 'indices': [227, 236]}, {'text': 'corona', 'indices': [237, 244]}, {'text': 'vascular', 'indices': [245, 254]}, {'text': 'endovascular', 'indices': [255, 268]}], 'userid': '1091660129894838272', 'username': 'Milli teknoloji', 'user_screen_name': 'milliteknoloj', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'timestamp_ms': '1587817302499', 'reply_to_tweeet_id': '1253768179517657088', 'reply_to_user_id': '1222639789', 'reply_to_user_name': 'VinceMcMahon'}
{'_

In [ ]:
#If no reply existss move to Quoted tweets section